In [1]:
import numpy as np
import xarray as xr

from hilde.trajectory import reader

%matplotlib inline

In [2]:
trajectory = reader('trajectory.son.bz2')
traj = trajectory

[trajectory]   Parse trajectory
[son] read file:  trajectory.son.bz2
[trajectory]   .. done in 0.371s|||||||||||||||||||||||  1001/1001


In [3]:
times = traj.times

positions = np.array([a.get_positions() for a in trajectory])

velocities = np.array([a.get_velocities() for a in trajectory])

forces = np.array([a.get_forces() for a in trajectory])

e_kin = np.array([a.get_kinetic_energy() for a in trajectory])

temperatures = np.array([a.get_temperature() for a in trajectory])

symbols = traj[0].get_chemical_symbols()

masses = traj[0].get_masses()

ref_positions = traj[0].get_positions()

In [4]:
vec_index = ['time', 'atom', 'i']
scalar_index = 'time'

DS = xr.Dataset({
    'positions': (vec_index, positions),
    'velocities': (vec_index, velocities),
    'forces': (vec_index, forces),
    'kinetic energy': (scalar_index, e_kin),
    'temperature': (scalar_index, temperatures),
    },
    coords={'time': times},
    attrs={'symbols': symbols, 'masses': masses.tolist(), 'reference positions': ref_positions.flatten().tolist()})

In [5]:
type(masses.tolist())

list

In [6]:
DS

<xarray.Dataset>
Dimensions:         (atom: 8, i: 3, time: 1001)
Coordinates:
  * time            (time) float64 0.0 5.0 10.0 ... 4.99e+03 4.995e+03 5e+03
Dimensions without coordinates: atom, i
Data variables:
    positions       (time, atom, i) float64 0.0 0.0 0.0 ... 4.043 4.045 1.352
    velocities      (time, atom, i) float64 0.004332 -0.02823 ... -0.004992
    forces          (time, atom, i) float64 -0.0 -0.0 -0.0 ... 0.3374 0.08589
    kinetic energy  (time) float64 0.3102 0.2725 0.1836 ... 0.1225 0.1885 0.2445
    temperature     (time) float64 300.0 263.5 177.6 95.84 ... 118.4 182.3 236.4
Attributes:
    symbols:              ['Si', 'Si', 'Si', 'Si', 'Si', 'Si', 'Si', 'Si']
    masses:               [28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28...
    reference positions:  [0.0, 0.0, 0.0, 0.0, 2.71303376291, 2.71303376291, ...

### Problem: global attrs cannot be written to netCDF files

In [7]:
try:
    DS.to_netcdf('test.nc')
except ValueError as error:
    print('netcdf complains')

### Solution: Save attrs to each DataArray within Dataset

In [8]:
das = {}
for key in DS:
    da = DS[key]
    da.attrs = DS.attrs
    das[key] = da

new_DS = xr.Dataset(das)

In [9]:
new_DS.to_netcdf('test.nc')

In [10]:
read_DS = xr.load_dataset('test.nc')

In [11]:
read_DS.attrs

OrderedDict()

In [12]:
read_DS.positions.attrs

OrderedDict([('symbols', ['Si', 'Si', 'Si', 'Si', 'Si', 'Si', 'Si', 'Si']),
             ('masses',
              array([28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085])),
             ('reference positions',
              array([0.        , 0.        , 0.        , 0.        , 2.71303376,
                     2.71303376, 2.71303376, 0.        , 2.71303376, 2.71303376,
                     2.71303376, 0.        , 1.35651688, 1.35651688, 1.35651688,
                     1.35651688, 4.06955064, 4.06955064, 4.06955064, 1.35651688,
                     4.06955064, 4.06955064, 4.06955064, 1.35651688]))])